### ETL - Extract, Transform, Load data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
torch.set_grad_enabled(True)
from torch.utils.tensorboard import SummaryWriter

In [2]:
train_set = torchvision.datasets.FashionMNIST(
    root='../data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [3]:
import torch.nn as nn

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # 1 input 
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        return t

In [4]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
# The best part about TensorBoard is its out-of-the-box capability of tracking 
# our hyperparameters over time and across runs.

#### TensorBoard Histograms And Scalars

In [7]:
parameters = dict(
    lr = [.01, .001]
    ,batch_size = [100, 1000]
    ,shuffle = [True, False]
)

param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [100, 1000], [True, False]]

In [9]:
from itertools import product

In [11]:
# example of a complete loop
network = Network()

for lr, batch_size, shuffle in product(*param_values): 
    comment = f' batch_size={batch_size} lr={lr} shuffle={shuffle}'
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
    optimizer = optim.Adam(network.parameters(), lr=lr)

    images, labels = next(iter(train_loader))
    grid = torchvision.utils.make_grid(images)

    tb = SummaryWriter()
    tb.add_image('images', grid)
    tb.add_graph(network, images)

    for epoch in range(3):

        total_loss = 0
        total_correct = 0

        for batch in train_loader: # Get Batch

            images, labels = batch 
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss

            optimizer.zero_grad() # this is used because the weights would be accumulated if not reset to zero
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            total_loss += loss.item()* images.shape[0] # 1
            total_correct += get_num_correct(preds, labels)

        tb.add_scalar('Loss', total_loss, epoch)
        tb.add_scalar('Number Correct', total_correct, epoch)
        tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

        for name, weight in network.named_parameters():
            tb.add_histogram(name, weight, epoch)
            tb.add_histogram(f'{name}.grad', weight.grad, epoch)

        print(
            "epoch", epoch, 
            "total_correct:", total_correct, 
            "loss:", total_loss
        )

tb.close()

epoch 0 total_correct: 46196 loss: 35514.61341679096
epoch 1 total_correct: 51020 loss: 24499.457749724388
epoch 2 total_correct: 51877 loss: 22003.507828712463
epoch 0 total_correct: 52089 loss: 21535.288953781128
epoch 1 total_correct: 52514 loss: 20343.014292418957
epoch 2 total_correct: 52736 loss: 19872.757725417614
epoch 0 total_correct: 53688 loss: 16914.28190469742
epoch 1 total_correct: 54034 loss: 15656.790286302567
epoch 2 total_correct: 54240 loss: 15146.981626749039
epoch 0 total_correct: 54234 loss: 15078.522741794586
epoch 1 total_correct: 54499 loss: 14334.07486975193
epoch 2 total_correct: 54618 loss: 14164.20367360115
epoch 0 total_correct: 54966 loss: 12930.421817302704
epoch 1 total_correct: 55124 loss: 12544.62364912033
epoch 2 total_correct: 55178 loss: 12317.50683709979
epoch 0 total_correct: 55223 loss: 12145.948259532452
epoch 1 total_correct: 55334 loss: 11961.580868065357
epoch 2 total_correct: 55398 loss: 11788.014801591635
epoch 0 total_correct: 55422 loss:

In [ ]:
# 1 For every batch, we are using the batch_size variable to update the total_loss value. 
# 1 We are scaling up the average loss value of the samples in the batch by the batch_size value. 
# 1 However, as we have just discussed, sometimes the last batch will contain fewer samples. 
# 1 Thus, scaling by the predefined batch_size value is inaccurate.